# Скрипт для запуска NAB-бенчмарка внутри jupyter

Чтобы изменить модель, необходимо отредактировать путь к `.h5`-файлу в `nab/detectors/my/my_detector.py`

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
#!/usr/bin/env python
# Copyright 2014-2015 Numenta Inc.
#
# Copyright may exist in Contributors' modifications
# and/or contributions to the work.
#
# Use of this source code is governed by the MIT
# license that can be found in the LICENSE file or at
# https://opensource.org/licenses/MIT.

!rm -rf `find -type d -name .ipynb_checkpoints`

import argparse
import os
try:
  import simplejson as json
except ImportError:
  import json

from nab.runner import Runner
from nab.util import (detectorNameToClass, checkInputs)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def getDetectorClassConstructors(detectors):
  """
  Takes in names of detectors. Collects class names that correspond to those
  detectors and returns them in a dict. The dict maps detector name to class
  names. Assumes the detectors have been imported.
  """
  detectorConstructors = {
  d : globals()[detectorNameToClass(d)] for d in detectors}

  return detectorConstructors


def main(args):

  root = os.path.realpath(os.path.abspath(''))

  numCPUs = int(args.numCPUs) if args.numCPUs is not None else None

  dataDir = os.path.join(root, args.dataDir)
  windowsFile = os.path.join(root, args.windowsFile)
  resultsDir = os.path.join(root, args.resultsDir)
  profilesFile = os.path.join(root, args.profilesFile)
  thresholdsFile = os.path.join(root, args.thresholdsFile)

  runner = Runner(dataDir=dataDir,
                  labelPath=windowsFile,
                  resultsDir=resultsDir,
                  profilesPath=profilesFile,
                  thresholdPath=thresholdsFile,
                  numCPUs=numCPUs)

  runner.initialize()

  if args.detect:
    detectorConstructors = getDetectorClassConstructors(args.detectors)
    runner.detect(detectorConstructors)

  if args.optimize:
    runner.optimize(args.detectors)

  if args.score:
    with open(args.thresholdsFile) as thresholdConfigFile:
      detectorThresholds = json.load(thresholdConfigFile)
    runner.score(args.detectors, detectorThresholds)

  if args.normalize:
    try:
      runner.normalize()
    except AttributeError("Error: you must run the scoring step with the "
                          "normalization step."):
      return


if __name__ == "__main__":

  parser = argparse.ArgumentParser()

  parser.add_argument("--detect",
                    help="Generate detector results but do not analyze results "
                    "files.",
                    default=False,
                    action="store_true")

  parser.add_argument("--optimize",
                    help="Optimize the thresholds for each detector and user "
                    "profile combination",
                    default=False,
                    action="store_true")

  parser.add_argument("--score",
                    help="Analyze results in the results directory",
                    default=False,
                    action="store_true")

  parser.add_argument("--normalize",
                    help="Normalize the final scores",
                    default=False,
                    action="store_true")

  parser.add_argument("--skipConfirmation",
                    help="If specified will skip the user confirmation step",
                    default=False,
                    action="store_true")

  parser.add_argument("--dataDir",
                    default="data",
                    help="This holds all the label windows for the corpus.")

  parser.add_argument("--resultsDir",
                    default="results",
                    help="This will hold the results after running detectors "
                    "on the data")

  parser.add_argument("--windowsFile",
                    default=os.path.join("labels", "combined_windows.json"),
                    help="JSON file containing ground truth labels for the "
                         "corpus.")

  parser.add_argument("-d", "--detectors",
                    nargs="*",
                    type=str,
                    default=["null", "random",
                             "bayesChangePt", "windowedGaussian", "expose",
                             "relativeEntropy", "earthgeckoSkyline"],
                    help="Comma separated list of detector(s) to use, e.g. "
                         "null, expose")

  parser.add_argument("-p", "--profilesFile",
                    default=os.path.join("config", "profiles.json"),
                    help="The configuration file to use while running the "
                    "benchmark.")

  parser.add_argument("-t", "--thresholdsFile",
                    default=os.path.join("config", "thresholds.json"),
                    help="The configuration file that stores thresholds for "
                    "each combination of detector and username")

  parser.add_argument("-n", "--numCPUs",
                    default=None,
                    help="The number of CPUs to use to run the "
                    "benchmark. If not specified all CPUs will be used.")

  args = parser.parse_args(['-d', 'my', '--skipConfirmation'])

  if (not args.detect
      and not args.optimize
      and not args.score
      and not args.normalize):
    args.detect = True
    args.optimize = True
    args.score = True
    args.normalize = True

  if len(args.detectors) == 1:
    # Handle comma-seperated list argument.
    args.detectors = args.detectors[0].split(",")

  # The following imports are necessary for getDetectorClassConstructors to
  # automatically figure out the detector classes.
  # Only import detectors if used so as to avoid unnecessary dependency.
  if "bayesChangePt" in args.detectors:
    from nab.detectors.bayes_changept.bayes_changept_detector import (
      BayesChangePtDetector)
  if "null" in args.detectors:
    from nab.detectors.null.null_detector import NullDetector
  if "random" in args.detectors:
    from nab.detectors.random.random_detector import RandomDetector
  # By default the skyline detector is disabled, it can still be added to the
  # detectors argument to enable it, for more info see #335 and #333
  if "skyline" in args.detectors:
    from nab.detectors.skyline.skyline_detector import SkylineDetector
  if "my" in args.detectors:
    from nab.detectors.my.my_detector import MyDetector
  if "windowedGaussian" in args.detectors:
    from nab.detectors.gaussian.windowedGaussian_detector import (
      WindowedGaussianDetector)
  if "knncad" in args.detectors:
    from nab.detectors.knncad.knncad_detector import KnncadDetector
  if "relativeEntropy" in args.detectors:
    from nab.detectors.relative_entropy.relative_entropy_detector import (
      RelativeEntropyDetector)
  if "expose" in args.detectors:
    from nab.detectors.expose.expose_detector import ExposeDetector
  if "contextOSE" in args.detectors:
    from nab.detectors.context_ose.context_ose_detector import (
    ContextOSEDetector )
  if "earthgeckoSkyline" in args.detectors:
    from nab.detectors.earthgecko_skyline.earthgecko_skyline_detector import EarthgeckoSkylineDetector
  if "ARTime" in args.detectors:
    from nab.detectors.ARTime.ARTime_detector import ARTimeDetector

  if args.skipConfirmation or checkInputs(args):
    main(args)



Running detection step


2025-05-12 17:11:31.217189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-12 17:12:23.105676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-12 17:12:23.187134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-12 17:12:23.311436: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-12 17:12:23.409121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-12 17:12:23.575233: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ER

0: Beginning detection with my for artificialNoAnomaly/art_daily_no_noise.csv
2: Beginning detection with my for artificialNoAnomaly/art_daily_small_noise.csv
4: Beginning detection with my for artificialNoAnomaly/art_noisy.csv
6: Beginning detection with my for artificialWithAnomaly/art_daily_jumpsdown.csv
8: Beginning detection with my for artificialWithAnomaly/art_daily_nojump.csv
10: Beginning detection with my for artificialWithAnomaly/art_load_balancer_spikes.csv
14: Beginning detection with my for realAWSCloudwatch/ec2_cpu_utilization_77c1ca.csv
12: Beginning detection with my for realAWSCloudwatch/ec2_cpu_utilization_53ea38.csv
125/125 [==============================] - 7s 33ms/step
0: Completed processing 4032 records at 2025-05-12 17:12:34.384654
0: Results have been written to /home/jupyter/work/resources/metrics-anomaly-detection/src/ml/NAB/results/my/artificialNoAnomaly/my_art_daily_no_noise.csv
1: Beginning detection with my for artificialNoAnomaly/art_daily_perfect_squar